In [19]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed
from tensorflow.keras import layers

from util import read_protein_file, split_based_on_windows, ohe_for_nn, convert_pred_to_str, create_plot

tf.keras.utils.set_random_seed(812)  # sets seeds for base-python, numpy and tf
tf.config.experimental.enable_op_determinism()          
os.environ['TF_DETERMINISTIC_OPS'] = '1'   

In [20]:
WINDOW_SIZE = 17
protein_letters='ACDEFGHIKLMNPQRSTVWXY'
secondary_letters='ceh'

In [21]:
# Preprocess data
train_seq, train_str = read_protein_file(
    "data/protein-secondary-structure.train")
test_seq, test_str = read_protein_file(
    "data/protein-secondary-structure.test")

train_df = split_based_on_windows(train_seq, train_str, WINDOW_SIZE)
test_df = split_based_on_windows(test_seq, test_str, WINDOW_SIZE)

X_train, y_train = ohe_for_nn(train_df['sequence'], train_df['string'])
X_test, y_test = ohe_for_nn(test_df['sequence'], test_df['string'])

In [22]:
@tf.keras.utils.register_keras_serializable(package="Custom", name="q3_score")
def q3_score(target, prediction):
    target_labels = tf.argmax(target, axis=-1)
    prediction_labels = tf.argmax(prediction, axis=-1)
    q3 = tf.reduce_mean(
        tf.cast(tf.equal(target_labels, prediction_labels), dtype=tf.float32))
    return q3

In [23]:
def model_builder(hp):
    hp_dropout = hp.Float('dropout', 0.1, 0.9, 0.1)
    hp_units = hp.Int('units', min_value=16, max_value=128, step=16)
    hp_units2 = hp.Int('units2', min_value=16, max_value=128, step=16)
    hp_activation = hp.Choice('activation', ["relu", "sigmoid", "tanh"])


    model = Sequential(
        [
            LSTM(
                hp_units,
                input_shape=(WINDOW_SIZE, len(protein_letters)),
                return_sequences=True,
            ),
            layers.BatchNormalization(),
            layers.Dropout(hp_dropout),
            Dense(hp_units2, activation=hp_activation),
            TimeDistributed(Dense(len(secondary_letters), activation="softmax")),
        ]
    )

    hp_learning_rate = hp.Float('learning_rate', 1e-3, 1.0, 1e-3)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy", "mae", q3_score],
    )

    return model

In [24]:
objective = kt.Objective("val_q3_score", direction="max")

tuner = kt.Hyperband(
    model_builder,
    objective=objective,
    max_epochs=50,
    factor=3,
    directory="tuning",
    project_name="nn_model",
)

In [25]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2)
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

best_hps_values = best_hps.values

for param, value in best_hps_values.items():
    print(f"{param}: {value}")

Trial 90 Complete [00h 00m 17s]
val_q3_score: 0.5579157471656799

Best val_q3_score So Far: 0.5818808674812317
Total elapsed time: 00h 09m 08s
dropout: 0.4
units: 32
units2: 96
activation: tanh
learning_rate: 0.051000000000000004
tuner/epochs: 50
tuner/initial_epoch: 17
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0068


In [26]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=100, validation_split=0.2)

val_acc_per_epoch = history.history['val_q3_score']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
24/24 [==============================] - 3s 30ms/step - loss: 1.0796 - accuracy: 0.5137 - mae: 0.3785 - q3_score: 0.5186 - val_loss: 1.0371 - val_accuracy: 0.5580 - val_mae: 0.3447 - val_q3_score: 0.5579
Epoch 2/100
24/24 [==============================] - 0s 8ms/step - loss: 0.9659 - accuracy: 0.5649 - mae: 0.3602 - q3_score: 0.5642 - val_loss: 1.0002 - val_accuracy: 0.5580 - val_mae: 0.3605 - val_q3_score: 0.5579
Epoch 3/100
24/24 [==============================] - 0s 9ms/step - loss: 0.9254 - accuracy: 0.5756 - mae: 0.3511 - q3_score: 0.5750 - val_loss: 0.9506 - val_accuracy: 0.5580 - val_mae: 0.3832 - val_q3_score: 0.5579
Epoch 4/100
24/24 [==============================] - 0s 9ms/step - loss: 0.9133 - accuracy: 0.5760 - mae: 0.3489 - q3_score: 0.5771 - val_loss: 1.0071 - val_accuracy: 0.5580 - val_mae: 0.3463 - val_q3_score: 0.5579
Epoch 5/100
24/24 [==============================] - 0s 11ms/step - loss: 0.8944 - accuracy: 0.5914 - mae: 0.3421 - q3_score: 0.5897 - val_

In [27]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/12
24/24 [==============================] - 3s 28ms/step - loss: 1.0923 - accuracy: 0.5058 - mae: 0.3804 - q3_score: 0.5108 - val_loss: 1.0868 - val_accuracy: 0.5580 - val_mae: 0.3484 - val_q3_score: 0.5579
Epoch 2/12
24/24 [==============================] - 0s 7ms/step - loss: 0.9444 - accuracy: 0.5700 - mae: 0.3543 - q3_score: 0.5702 - val_loss: 0.9637 - val_accuracy: 0.5580 - val_mae: 0.3562 - val_q3_score: 0.5579
Epoch 3/12
24/24 [==============================] - 0s 8ms/step - loss: 0.9238 - accuracy: 0.5774 - mae: 0.3511 - q3_score: 0.5749 - val_loss: 0.9597 - val_accuracy: 0.5580 - val_mae: 0.3814 - val_q3_score: 0.5579
Epoch 4/12
24/24 [==============================] - 0s 8ms/step - loss: 0.9183 - accuracy: 0.5749 - mae: 0.3515 - q3_score: 0.5756 - val_loss: 0.9801 - val_accuracy: 0.5580 - val_mae: 0.3486 - val_q3_score: 0.5579
Epoch 5/12
24/24 [==============================] - 0s 9ms/step - loss: 0.8987 - accuracy: 0.5904 - mae: 0.3431 - q3_score: 0.5896 - val_loss: 

In [28]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

6/6 [==============================] - 0s 3ms/step - loss: 0.9124 - accuracy: 0.5721 - mae: 0.3511 - q3_score: 0.5722
[test loss, test accuracy]: [0.9123649001121521, 0.5721362233161926, 0.35105791687965393, 0.5722426176071167]
